In [1]:
import pandas as pd
from os import listdir

In [2]:
# Coletando nomes dos arquivo CSV presentes na pasta "path_to_dir"
path_to_dir = "data/BDTD_feb2025"
filenames = listdir(path_to_dir)
csv_files = [ path_to_dir + "/" + filename for filename in filenames if filename.endswith( ".csv" ) ]

In [3]:
# Concatenando todos os arquivos
data = pd.DataFrame()
for csv in csv_files:
    data = pd.concat([data, pd.read_csv(csv)])
data = data.drop_duplicates(subset=['Título'], keep='first').reset_index(drop=True)

In [4]:
# Quebrando assuntos em lista
data["Assuntos em português"] = data["Assuntos em português"].apply(lambda x: x.split("||"))
data["Assuntos em inglês"] = data["Assuntos em inglês"].apply(lambda x: x.split("||"))

In [5]:
# Limpando o nome dos autores
autors = data["Autor(a)"].apply(lambda x: x.split("||"))

new_autors = []
for n in range(len(autors)):
    new_autors.append(autors[n][0])
    if len(autors[n]) > 1:
        if autors[n][1].startswith("http://lattes.cnpq.br/"):
            data.loc[n, "ID Lattes do(a) autor(a)"] = autors[n][1]

data["Autor(a)"] = new_autors

In [6]:
# Limpando os nomes dos orientadores

orientadores = data["Orientadores"].apply(lambda x: x.split("||"))

for n in range(len(orientadores)):
    if "Não informado pela instituição" in orientadores[n]:
        orientadores[n] = list(filter(lambda a: a != "Não informado pela instituição", orientadores[n]))

data["Orientadores"] = orientadores

In [7]:
# Limpando os nomes dos membros da banca

banca = data["Membros da banca"].apply(lambda x: x.split("||"))

for n in range(len(banca)):
    if "Não informado pela instituição" in banca[n]:
        banca[n] = list(filter(lambda a: a != "Não informado pela instituição", banca[n]))

data["Membros da banca"] = banca

In [8]:
# Limpando os textos do Resumo
data['Resumo'] = data['Resumo'].astype(str)
data[data['Resumo'].str.contains('Orientador:')] = ''
data[data['Resumo'].str.contains('Orientadora:')] = ''
data[data['Resumo'].str.contains('Orientadores:')] = ''
data[data['Resumo'] == '--'] = ''
data[data['Resumo'] == '.'] = ''
data[data['Resumo'].str.contains(r'Dissertação \(mestrado\) -', regex=True)] = ''
data[data['Resumo'].str.contains('também disponível em formato')] = ''
data[data['Resumo'] == '[sem resumo]'] = ''
data[data['Resumo'] == 'abstract'] = ''
data[data['Resumo'].str.contains('Tese \(doutorado\)')] = ''
data[data['Resumo'] == 'Não informado pela instituição'] = ''

C:\Users\facordei\AppData\Local\Temp\ipykernel_13168\147117010.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data[data['Resumo'].str.contains('Orientador:')] = ''


Filtrar as áreas de conhecimento e assuntos indesejados

In [92]:
AC_assunt = data[['Área do conhecimento CNPq', 'Assuntos em português', 'Assuntos em inglês']]
AC_assunt['Área do conhecimento CNPq'] = AC_assunt['Área do conhecimento CNPq'].apply(lambda x: x.split('||'))
AC_assunt['Área do conhecimento CNPq'] = AC_assunt['Área do conhecimento CNPq'].apply(lambda x:  [a.split('::') if '::' in a  else a for a in x] )
AC_assunt['Área do conhecimento CNPq'] = AC_assunt['Área do conhecimento CNPq'].apply(lambda x: list(set([item.lower()  for sublist in x for item in (sublist if isinstance(sublist, list) else [sublist])])))
AC_assunt['Assuntos em português'] = AC_assunt['Assuntos em português'].apply(lambda x: [a.lower() for a in x])
AC_assunt['Assuntos em inglês'] = AC_assunt['Assuntos em inglês'].apply(lambda x: [a.lower() for a in x])

C:\Users\facordei\AppData\Local\Temp\ipykernel_13168\3699077921.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AC_assunt['Área do conhecimento CNPq'] = AC_assunt['Área do conhecimento CNPq'].apply(lambda x: x.split('||'))
C:\Users\facordei\AppData\Local\Temp\ipykernel_13168\3699077921.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AC_assunt['Área do conhecimento CNPq'] = AC_assunt['Área do conhecimento CNPq'].apply(lambda x:  [a.split('::') if '::' in a  else a for a in x] )
C:\Users\facordei\AppD

In [97]:
# Area do conheciumento CNPq
#data[AC_assunt['Área do conhecimento CNPq'].apply(lambda x : 'matematica' in x)][['Título', 'Assuntos em português', 'Assuntos em inglês']]

# Assuntos em português
#data[AC_assunt['Assuntos em português'].apply(lambda x : 'slave' in x)][['Título', 'Assuntos em português', 'Assuntos em inglês']]

# Assuntos em inglês
data[AC_assunt['Assuntos em inglês'].apply(lambda x : 'slave' in x)][['Título', 'Assuntos em português', 'Assuntos em inglês']]

,Título,Assuntos em português,Assuntos em inglês
67,A argumentação na obra O abolicionismo de Joaq...,"[Escravo, Liberdade, Direito, Justiça, Consciê...","[Slave, Freedom, Right, Justice, National cons..."
362,Olhar para além das enfermidades: ser liberto ...,"[liberto, movimento, abolição, Escravos - Cear...","[slave, I free, I move, abolition.]"
6465,Cotidiano em conflito: relações sociais e fami...,"[Divorcio, Matrimônio, Sociabilidade, Escravos...","[Woman, Divorce, Colony, Slave, Marriage, Soci..."
7025,Uma análise da lista suja no Maranhão como açã...,"[Trabalho, Escravo, Lista suja, Maranhão]","[Work, Slave, Dirty list]"
7040,Trabalho escravo contemporâneo no Maranhão: um...,"[Trabalho, Escravo, Maranhão, Erradicação, Lis...","[Work, Slave, Maranhão, Eradication, Dirty Lis..."
7087,Os sentidos de escravo em Memórias Póstumas de...,"[Semântica, Designação, Machado, Escravo, Acon...","[Semantics, Designation, Slave, Literature, Ev..."
7135,"""Servindo a Deus e ao rei"": escravidão velada,...","[Diretório, Tutela, Liberdade dos índios, Escr...","[Indian chiefs, Directory, Guardianship, Freed..."
7474,A liberdade em Aristóteles,"[Liberdade, Razão, Alma, Escravo, Lei]","[Freedom, Reason, Soul, Slave, Law]"


In [ ]:
# Excluindo as áreas não informadas pela instituicão
AC = data['Área do conhecimento CNPq'][ data['Área do conhecimento CNPq'] != 'Não informado pela instituição']
#Identificando as sequencias de areas únicas (todo um ramo da taxonomia)
AC = AC.apply(lambda x: x.split('||'))
AC_list = []
for area in AC:
    AC_list = AC_list + area
AC_list = list(set(AC_list))

In [9]:
# Salvando como csv
path_to_dir_output = "data"
data.to_csv(path_to_dir_output + "/BDTD_Consolidado_feb2025.csv", index=False)